In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

In [4]:
df_geographic_unique = df_geographic_unique.dropna(subset=['latitude', 'censusBlockGroupFips'])

In [5]:
df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

In [6]:
# Define bins and labels for yearOfLoss_1980_2020
bins_1980_2020 = [df_geographic_unique['yearOfLoss'].min(), 1990, 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1980_2020 = [1980, 1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1980_2020'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1980_2020, labels=labels_1980_2020, right=False).astype(int)

In [ ]:
# states = pygris.states()
# states_list = sorted(states['STUSPS'].unique())

In [ ]:
# BG_list_2000 = []

# for i in states_list:
#     BG_list_2000.append(pygris.block_groups(state = i, year = 2000))

In [7]:
BG_list_df2000 = pygris.block_groups(state = 'NC', year = 2000)

Using FIPS code '37' for input 'NC'


In [ ]:
# BG_list_1990 = []

# for i in states_list:
#     BG_list_1990.append(pygris.block_groups(state = i, year = 1990, cb = True))

In [8]:
BG_list_df1990 = pygris.block_groups(state = 'NC', year = 1990, cb = True)

Using FIPS code '37' for input 'NC'


In [ ]:
# BG_list_df2000 = pd.concat(BG_list_2000, ignore_index=True)
# BG_list_df1990 = pd.concat(BG_list_1990, ignore_index=True)

In [9]:
BG_list_df2000 = BG_list_df2000[['BKGPIDFP00', 'geometry']]
BG_list_df2000['year'] = 2000

BG_list_df2000 = BG_list_df2000.rename(columns={"BKGPIDFP00": "GEOID"})

BG_list_df1990 = BG_list_df1990[['GEOID', 'geometry']]
BG_list_df1990['year'] = 1990

#### Checking for 2000...

In [ ]:
# df_geographic_unique2000 = df_geographic_unique[df_geographic_unique['yearOfLoss_1980_2020']==2000]

In [10]:
df_geographic_unique2000 = df_geographic_unique[(df_geographic_unique['yearOfLoss_1980_2020']==2000) & (df_geographic_unique['state']=='NC')]

In [11]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [12]:
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)

In [13]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique2000['geometry_lat_long'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_30356\408559743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geographic_unique2000['geometry_lat_long'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])


In [14]:
import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [18]:
df_geographic_unique2000_lat_long = df_geographic_unique2000[['latitude', 'longitude', 'geometry_lat_long']].drop_duplicates()

In [20]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(BG_list_df2000, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (Web Mercator in this case)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the area for each geometry in square kilometers
gdf_projected['area_km2'] = gdf_projected.geometry.area / 1e6

# Filter out geometries with area less than 152.1611272703287 square kilometers
filtered_gdf = gdf_projected[gdf_projected['area_km2'] >= 152.16]

# If you want the filtered data in the original CRS, you can do the following:
BG_list_df2000_filtered = filtered_gdf.to_crs(epsg=4326)

In [55]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df2000 = gpd.GeoDataFrame(columns=['MATCHED_censusBlockGroupFips', 'latitude', 'longitude', 'geometry_BG',
                                        'geometry_lat_long', 'new_geometry'])

# Iterate through each row in BG_df_2000 and each row in lat_long_df to find intersections
for idx_unit_data, row_unit_data in df_geographic_unique2000_lat_long.iterrows():
    for idx_unit_shape, row_unit_shape in BG_list_df2000_filtered.iterrows():

        bg_id2 = row_unit_shape['GEOID']
        bg_geometry = row_unit_shape['geometry']
        lat_long_geometry = row_unit_data['geometry_lat_long']
        lat = row_unit_data['latitude']
        long = row_unit_data['longitude']
    
        # Compute intersection geometry
        intersection_geometry = bg_geometry.intersection(lat_long_geometry)
        
        intersection_gseries = gpd.GeoSeries([intersection_geometry], crs="EPSG:4326")
        intersection_gseries_projected = intersection_gseries.to_crs(epsg=3857)
        area_m2 = intersection_gseries_projected.iloc[0].area
        
        if area_m2 / 1e6 >= 100.1611272703287:
    
            new_unit_df2000 = pd.concat([new_unit_df2000, pd.DataFrame({
                'MATCHED_censusBlockGroupFips': [bg_id2],
                'latitude': [lat],
                'longitude': [long],
                'geometry_BG': [bg_geometry],
                'geometry_lat_long': [lat_long_geometry],
                'new_geometry': [intersection_geometry]
            })], ignore_index=True)

In [57]:
new_unit_df2000.shape[0]

8

In [58]:
# Setting the multi-index on filtered_df
new_unit_df2000.set_index(['latitude', 'longitude'], inplace=True)

# Mapping values one by one
df_geographic_unique2000['MATCHED_censusBlockGroupFips'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(new_unit_df2000['MATCHED_censusBlockGroupFips'])
df_geographic_unique2000['geometry_BG'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(new_unit_df2000['geometry_BG'])
df_geographic_unique2000['new_geometry'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(new_unit_df2000['new_geometry'])

# Assuming lat_long_df should be df_geographic_unique2000 based on context
new_unit_df2000.reset_index(inplace=True)

In [59]:
df_geographic_unique2000.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_1980_2020,geometry_lat_long,MATCHED_censusBlockGroupFips,geometry_BG,new_geometry
31,NC,28646.0,37011.0,3.701193e+10,370119302003,36.0,-81.9,2004,2000,"POLYGON ((-81.95000 35.95000, -81.85000 35.950...",NaN,NaN,NaN
90,NC,27920.0,37055.0,3.705597e+10,370559705022,35.3,-75.5,2003,2000,"POLYGON ((-75.55000 35.25000, -75.45000 35.250...",NaN,NaN,NaN
185,NC,27949.0,37055.0,3.705597e+10,370559701022,36.1,-75.7,2003,2000,"POLYGON ((-75.75000 36.05000, -75.65000 36.050...",NaN,NaN,NaN
235,NC,27927.0,37053.0,3.705311e+10,370531101012,36.4,-75.8,2006,2000,"POLYGON ((-75.85000 36.35000, -75.75000 36.350...",NaN,NaN,NaN
256,NC,27909.0,37139.0,3.713996e+10,371399605021,36.3,-76.2,2006,2000,"POLYGON ((-76.25000 36.25000, -76.15000 36.250...",NaN,NaN,NaN


In [61]:
filtered_df = df_geographic_unique2000[df_geographic_unique2000['MATCHED_censusBlockGroupFips'].notna()]

In [66]:
filtered_df[filtered_df['censusBlockGroupFips']!=filtered_df['MATCHED_censusBlockGroupFips']]

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_1980_2020,geometry_lat_long,MATCHED_censusBlockGroupFips,geometry_BG,new_geometry
5407,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.7,2003,2000,"POLYGON ((-75.75000 35.65000, -75.65000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.75 35.75, -75.65 35.75, -75.65 3..."
14690,NC,27824.0,37095.0,3.709592e+10,370959201003,35.5,-76.0,2003,2000,"POLYGON ((-76.05000 35.45000, -75.95000 35.450...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.05 35.55, -75.95 35.55, -75.95 3..."
241614,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.7,2004,2000,"POLYGON ((-75.75000 35.65000, -75.65000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.75 35.75, -75.65 35.75, -75.65 3..."
403717,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.7,2008,2000,"POLYGON ((-75.75000 35.65000, -75.65000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.75 35.75, -75.65 35.75, -75.65 3..."
403718,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.7,2009,2000,"POLYGON ((-75.75000 35.65000, -75.65000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.75 35.75, -75.65 35.75, -75.65 3..."
550378,NC,27824.0,37095.0,3.709592e+10,370959201003,35.5,-76.0,2007,2000,"POLYGON ((-76.05000 35.45000, -75.95000 35.450...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.05 35.55, -75.95 35.55, -75.95 3..."
554301,NC,27978.0,37055.0,3.705597e+10,370559705011,35.7,-75.8,2003,2000,"POLYGON ((-75.85000 35.65000, -75.75000 35.650...",370559705004,"POLYGON ((-75.59184199999999 35.758117, -75.59...","POLYGON ((-75.85 35.75, -75.75 35.75, -75.75 3..."
624424,NC,27824.0,37095.0,3.709592e+10,370959201003,35.5,-76.1,2003,2000,"POLYGON ((-76.15000 35.45000, -76.05000 35.450...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.14999999999999 35.55, -76.05 35...."
625303,NC,27824.0,37095.0,3.709592e+10,370959201003,35.5,-76.0,2004,2000,"POLYGON ((-76.05000 35.45000, -75.95000 35.450...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.05 35.55, -75.95 35.55, -75.95 3..."
626412,NC,27824.0,37095.0,3.709592e+10,370959201003,35.6,-76.1,2003,2000,"POLYGON ((-76.15000 35.55000, -76.05000 35.550...",370959801001,"POLYGON ((-76.211888 35.46288299999999, -76.21...","POLYGON ((-76.14999999999999 35.65, -76.05 35...."


In [67]:
filtered_df.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_analysis.csv')

#### Checking for 1990

In [ ]:
# df_geographic_unique1990 = df_geographic_unique[df_geographic_unique['yearOfLoss_1980_2020']==1990]

In [ ]:
df_geographic_unique1990= df_geographic_unique[(df_geographic_unique['yearOfLoss_1980_2020']==1990) & (df_geographic_unique['state']=='NC')]

In [ ]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique1990['geometry_lat_long'] = df_geographic_unique1990.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

In [ ]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df1990 = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'MATCHED_censusBlockGroupFips', 'latitude', 'longitude', 'year', 'state', 'geometry_BG',
                                        'geometry_lat_long', 'new_geometry'])

# Iterate through each row in BG_df_2000 and each row in lat_long_df to find intersections
for idx_unit_data, row_unit_data in df_geographic_unique2000.iterrows():
    for idx_unit_shape, row_unit_shape in BG_list_df2000.iterrows():
        year = row_unit_data['yearOfLoss']
        year2 = row_unit_data['yearOfLoss_1980_2020']
        bg_id = row_unit_data['censusBlockGroupFips']
        bg_id2 = row_unit_shape['GEOID']
        bg_geometry = row_unit_shape['geometry']
        tract_id = row_unit_data['censusTract']
        county_id = row_unit_data['countyCode']
        state = row_unit_data['state']
        lat_long_geometry = row_unit_data['geometry_lat_long']
        lat = row_unit_data['latitude']
        long = row_unit_data['longitude']
        zipcode = row_unit_data['reportedZipCode']
    
        # Compute intersection geometry
        intersection_geometry = bg_geometry.intersection(lat_long_geometry)
    
        new_unit_df1990 = pd.concat([new_unit_df1990, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'MATCHED_censusBlockGroupFips': [bg_id2],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'decade': [year2],
            'state': [state],
            'geometry_BG': [bg_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'new_geometry': [intersection_geometry]
        })], ignore_index=True)